<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/qchem_400_TripleGivens_template/qchem_400_TripleGivens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
!pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np

[Givens rotations](https://pennylane.ai/qml/demos/tutorial_givens_rotations.html) are special particle preserving gates used in Quantum Chemistry. Given how the Single Excitation and Double Excitation Givens work, the goal in this challenge is to create a Triple Givens Excitation Gate and use them to create the circuit:

$$
|\psi ( \alpha, \beta, \gamma) \rangle = cos \frac{\alpha}{2} cos \frac{\beta}{2} \left[ cos \frac{\gamma}{2} |111000\rangle - sin \frac{\gamma}{2} |000111\rangle\right] - cos \frac{\alpha}{2} sin \frac{\beta}{2} |001011\rangle - sin \frac{\alpha}{2} |011001\rangle
$$

First, let's understand how to create the Triple Givens Excitation gate. Our requirements are the following:
 
$$
G^3(\theta) |111000\rangle = cos \frac{\theta}{2} |111000\rangle - sin \frac{\theta}{2} |000111\rangle \\
G^3(\theta) |000111\rangle = cos \frac{\theta}{2} |000111\rangle + sin \frac{\theta}{2} |111000\rangle
$$ 

It's easy to see waht we are only working with the states $|111000\rangle$ and $|000111\rangle$ or, using their decimal representation, 56 and 7.

Here's how we obtain this behaviour:

In [3]:
def triple_excitation_matrix(gamma):
    """The matrix representation of a triple-excitation Givens rotation.

    Args:
        - gamma (float): The angle of rotation

    Returns:
        - (np.ndarray): The matrix representation of a triple-excitation
    """

    # QHACK #
    # Generate coefficients
    c = qml.math.cos(gamma / 2)
    s = qml.math.sin(gamma / 2)
    # Start with a diagonal matrix
    mat = qml.math.diag([1.0] * (2 ** NUM_WIRES))
    # Decimal representations of states
    i, j = 7, 56
    # Alter matrix values that alter only the states 7 and 56
    mat[i, i] = c
    mat[i, j] = -s
    mat[j, i] = s
    mat[j, j] = c
    return mat

Now that we have the circuit, it is easy that we can obtain the desired state as follows:

1. Initialize the system in the state $|111000\rangle$:
$$
|\psi\rangle = |111000\rangle
$$

2. Apply a single excitation gate on qubits 0 and 5:
$$
|\psi (\alpha) \rangle = G¹_{0, 5}(\alpha) |\psi\rangle = G¹_{0, 5}(\alpha) |111000\rangle = cos \frac{\theta}{2} |111000\rangle - sin \frac{\theta}{2} |011001\rangle
$$

3. Apply a double excitation gate on qubits, 0, 1, 4 and 5:
$$
|\psi (\alpha, \beta) \rangle = G^2_{0, 1, 4, 5}(\beta) |\psi (\alpha) \rangle = G^2_{0, 1, 4, 5}(\beta) \left( cos \frac{\theta}{2} |111000\rangle - sin \frac{\theta}{2} |011001\rangle \right) = \\ 
cos \frac{\theta}{2} G^2_{0, 1, 4, 5}(\beta) |111000\rangle - sin \frac{\theta}{2} |011001\rangle = \\
cos \frac{\theta}{2} \left( cos \frac{\beta}{2} |111000\rangle  - sin \frac{\beta}{2} |001011\rangle \right) - sin \frac{\theta}{2} |011001\rangle = \\
cos \frac{\theta}{2} cos \frac{\beta}{2} |111000\rangle  - cos \frac{\theta}{2} sin \frac{\beta}{2} |001011\rangle - sin \frac{\theta}{2} |011001\rangle
$$

3. Apply the triple excitation gate on all qubits:
$$ 
|\psi (\alpha, \beta, \gamma) \rangle = G³(\gamma) |\psi (\alpha, \beta) \rangle = G³(\gamma) \left[ cos \frac{\theta}{2} cos \frac{\beta}{2} |111000\rangle  - cos \frac{\theta}{2} sin \frac{\beta}{2} |001011\rangle - sin \frac{\theta}{2} |011001\rangle \right] = \\
cos \frac{\theta}{2} cos \frac{\beta}{2} G³(\gamma) |111000\rangle  - cos \frac{\theta}{2} sin \frac{\beta}{2} |001011\rangle - sin \frac{\theta}{2} |011001\rangle = \\
cos \frac{\theta}{2} cos \frac{\beta}{2} \left[ cos \frac{\gamma}{2}|111000\rangle - sin \frac{\gamma}{2}|000111\rangle  \right] - cos \frac{\theta}{2} sin \frac{\beta}{2} |001011\rangle - sin \frac{\theta}{2} |011001\rangle
$$

Arriving at the desired state.

In [7]:
dev = qml.device("default.qubit", wires=6)
NUM_WIRES = 6

@qml.qnode(dev)
def circuit(angles):
    """Prepares the quantum state in the problem statement and returns qml.probs

    Args:
        - angles (list(float)): The relevant angles in the problem statement in this order:
        [alpha, beta, gamma]

    Returns:
        - (np.tensor): The probability of each computational basis state
    """
    # QHACK #
    alpha, beta, gamma = angles[0], angles[1], angles[2]
    # Initialize state
    qml.BasisState(np.array([1, 1, 1, 0, 0, 0]), wires=[0, 1, 2, 3, 4, 5])
    # SingleExcitation gate
    qml.SingleExcitation(alpha, wires=[0, 5])
    # DoubleExcitation gate
    qml.DoubleExcitation(beta, wires=[0, 1, 4, 5])
    # TripleExcitation gate from our implementation
    qml.QubitUnitary(triple_excitation_matrix(gamma), wires=[0, 1, 2, 3, 4, 5])
    # QHACK #
    return qml.probs(wires=range(NUM_WIRES))

Testing with 1.in

In [21]:
inputs = "2.71035258,2.86582337,4.46182774".split(",")
inputs = np.array(inputs, dtype=float)
probs = circuit(inputs).round(6)

# We interested in the states |111000> = 56, |000111> = 7, |001011> = 11 and |011001> = 25
a, b, g = inputs[0], inputs[1], inputs[2]
p_56 = probs[56]
p_7 = probs[7]
p_11 = probs[11]
p_25 = probs[25]
print(f"Measured prob |111000> {p_56} vs expected {(np.cos(a/2)*np.cos(b/2)*np.cos(g/2))**2}")
print(f"Measured prob |000111> {p_7} vs expected {(np.cos(a/2)*np.cos(b/2)*np.sin(g/2))**2}")
print(f"Measured prob |001011> {p_11} vs expected {(np.cos(a/2)*np.sin(b/2))**2}")
print(f"Measured prob |011001> {p_25} vs expected {(np.sin(a/2))**2}")

Measured prob |111000> 0.000325 vs expected 0.0003251869843297567
Measured prob |000111> 0.00054 vs expected 0.0005396118866232372
Measured prob |001011> 0.044911 vs expected 0.04491115090944501
Measured prob |011001> 0.954224 vs expected 0.9542240502196021


We can also test any random angle configuration

In [42]:
inputs = (np.random.rand(3)*2-1)*2*np.pi
probs = circuit(inputs).round(6)

# We interested in the states |111000> = 56, |000111> = 7, |001011> = 11 and |011001> = 25
a, b, g = inputs[0], inputs[1], inputs[2]
p_56 = probs[56]
p_7 = probs[7]
p_11 = probs[11]
p_25 = probs[25]
print(f"Measured prob |111000> {p_56} vs expected {(np.cos(a/2)*np.cos(b/2)*np.cos(g/2))**2}")
print(f"Measured prob |000111> {p_7} vs expected {(np.cos(a/2)*np.cos(b/2)*np.sin(g/2))**2}")
print(f"Measured prob |001011> {p_11} vs expected {(np.cos(a/2)*np.sin(b/2))**2}")
print(f"Measured prob |011001> {p_25} vs expected {(np.sin(a/2))**2}")

Measured prob |111000> 0.192114 vs expected 0.192114174145664
Measured prob |000111> 0.135426 vs expected 0.13542555314550161
Measured prob |001011> 0.580818 vs expected 0.5808182086967335
Measured prob |011001> 0.091642 vs expected 0.09164206401210083
